In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 

# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import os
import cv2

import tensorflow as tf
from sklearn.model_selection import train_test_split
from skimage.util import random_noise
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
path = '/kaggle/input/stanford-cars-dataset/cars_train/cars_train/'
filenames = list(glob.glob(path + '/*.jpg'))
print(filenames[:8])                 

### Test case for images:

In [ ]:
for i in range(9):
    plt.subplot(3, 3, i+1)
    image = cv2.imread(filenames[i])
    plt.imshow(image)

In [ ]:
train_img, val_img, _, _ = train_test_split(filenames, filenames, test_size = 0.2, shuffle = True, random_state = 0)

In [ ]:
print(f'Size of training set is: {len(train_img)}', end='\n----------------------------\n')
print(f'Size of validation set is: {len(val_img)}', end='\n----------------------------\n')
print(f'Total: {len(train_img) + len(val_img)}')

In [ ]:
WIDTH = 256
HEIGHT = 256
n_channels = 3
BATCH_SIZE = 64

In [ ]:
class  DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, orig_filenames, batch_size = BATCH_SIZE, shuffle = True):
        self.orig_filenames = orig_filenames
        self.noisy_filenames = orig_filenames
        self.filenames = list(zip(self.orig_filenames, self.noisy_filenames))
        self.batch_size = BATCH_SIZE
        self.shuffle = shuffle
    
    def __len__(self):
        return (len(self.orig_filenames) // self.batch_size)
    
    def __getitem__(self, idx):
        batch = self.filenames[idx * self.batch_size : (idx + 1) * self.batch_size]
        X, Y = self.__data_generation(batch)

        return X, Y
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.orig_filenames))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, batch):
        orig = []
        noisy = []
        
        for orig_file, _ in batch:
          # image = self.colored_image_loader(orig_file)
          image = cv2.imread(orig_file) / 255
#           image = tf.image.convert_image_dtype(image, tf.float32) / 255
          image = cv2.resize(image, (WIDTH, HEIGHT))
#           image = tf.image.convert_image_dtype(image, tf.float32)
            
          # noisy = cv2.imread(mask_file)
          # mask = cv2.resize(mask, (WIDTH, HEIGHT))[:, :, 2]

#           orig.append(cv2.resize(image, (252, 252)))
          orig.append(image)
          
          noisy.append(self.make_noisy(image))
          
        return  np.array(noisy), np.array(orig)

    # def colored_image_loader(self, image_filename):
    #   img = cv2.imread(image_filename)
    #   img = cv2.resize(img, (WIDTH, HEIGHT))
    #   img = tf.image.convert_image_dtype(img, tf.float32)
      
      # return img

    def make_noisy(self, image):
      # image = cv2.imread(img_file)
      image_ = image
      noisy_image = random_noise(image_, mode = 'gaussian')
      
      return noisy_image

### Testing the Data Generator:

In [ ]:
# data = train_X
data = DataGenerator(train_img, 64, True)
x, y = data[0]
# print(type(x))
# print(data[0].shape)
plt.imshow(x[31])
# plt.imshow(y[31])
# print(type(X))
# plt.imshow(X[1][2])
# print(X)
# print(train_X[:9])

In [ ]:
# class  DataGenerator(tf.keras.utils.Sequence):
    
#     def __init__(self, orig_filenames, batch_size = BATCH_SIZE, shuffle = True):

train_generator = DataGenerator(train_img, BATCH_SIZE)
val_generator = DataGenerator(val_img, BATCH_SIZE)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Conv2D, UpSampling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
# def create_model():
#   x = Input(shape=(WIDTH, HEIGHT, n_channels)) 

#   # Encoder
#   e_conv1 = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
#   pool1 = MaxPooling2D((2, 2), padding='same')(e_conv1)
#   batchnorm_1 = BatchNormalization()(pool1)
#   e_conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(batchnorm_1)
#   pool2 = MaxPooling2D((2, 2), padding='same')(e_conv2)
#   batchnorm_2 = BatchNormalization()(pool2)
#   e_conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(batchnorm_2)
#   h = MaxPooling2D((2, 2), padding='same')(e_conv3)


#   # Decoder
#   d_conv1 = Conv2D(256, (3, 3), activation='relu', padding='same')(h)
#   up1 = UpSampling2D((2, 2))(d_conv1)
#   d_conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(up1)
#   up2 = UpSampling2D((2, 2))(d_conv2)
#   d_conv3 = Conv2D(64, (3, 3), activation='relu')(up2)
#   up3 = UpSampling2D((2, 2))(d_conv3)
#   r = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(up3)

#   model = Model(x, r)
# #  model.compile(optimizer='adam', loss='mse', metrics = [''])
#   return model

In [ ]:
import tensorflow.keras.layers as tkl
def conv_block(inputs = None, n_filters = 32, kernel_size = 3, batch_norm = True):
    
    '''First layer'''
    
    conv = tkl.Conv2D(filters = n_filters,
                      kernel_size = kernel_size,
                      padding = 'same',
                      kernel_initializer = 'he_normal')(inputs)
    
    if batch_norm:
        conv = tkl.BatchNormalization()(conv)
    
    conv = tkl.Activation('relu')(conv)
    
    '''Second layer'''
    
    conv = tkl.Conv2D(filters = n_filters,
                     kernel_size = kernel_size,
                     padding = 'same',
                     kernel_initializer = 'he_normal')(conv)
    
    if batch_norm:
        conv = tkl.BatchNormalization()(conv)
    
    conv = tkl.Activation('relu')(conv)
        
    return conv


def conv_transpose_block(n_filters, kernel_size = 3):
    conv_transpose = tkl.Conv2DTranspose(n_filters, kernel_size = kernel_size, strides = (2,2), padding = 'same')
    
    return conv_transpose
    

def u_net(n_filters = 32, dropout_prob = 0.1, batch_norm = True):
    input_size = (WIDTH, HEIGHT, n_channels)
    
    input_img = tf.keras.Input(input_size, name = 'image' )
    
    c1 = conv_block(input_img, n_filters, kernel_size = 3)
    p1 = tkl.MaxPooling2D((2,2))(c1)
    p1 = tkl.Dropout(dropout_prob)(p1)
    
    c2 = conv_block(p1, n_filters * 2, kernel_size = 3)
    p2 = tkl.MaxPooling2D((2,2))(c2)
    p2 = tkl.Dropout(dropout_prob)(p2)
    
    c3 = conv_block(p2, n_filters * 4, kernel_size = 3)
    p3 = tkl.MaxPooling2D((2,2))(c3)
    p3 = tkl.Dropout(dropout_prob)(p3)
    
    c4 = conv_block(p3, n_filters * 8, kernel_size = 3)
    p4 = tkl.MaxPooling2D((2,2))(c4)
    p4 = tkl.Dropout(dropout_prob)(p4)
    
    c5 = conv_block(p4, n_filters * 16 , kernel_size = 3)
    
    ''' UpSampling '''
    
    u6 = conv_transpose_block(n_filters * 8, kernel_size = 3)(c5)  #----> 256
    u6 = tkl.concatenate([u6, c4])     #----> 512
    u6 = tkl.Dropout(dropout_prob)(u6)
    c6 = conv_block(u6, n_filters * 8, kernel_size = 3)   #----> 256
    
    u7 = conv_transpose_block(n_filters * 4)(c6)   #----> 128
    u7 = tkl.concatenate([u7, c3])   #----> 256
    u7 = tkl.Dropout(dropout_prob)(u7)
    c7 = conv_block(u7, n_filters * 4, kernel_size = 3)   #----> 128
    
    u8 = conv_transpose_block(n_filters * 2, kernel_size = 3)(c7)   #----> 64
    u8 = tkl.concatenate([u8, c2])   #----> 128
    u8 = tkl.Dropout(dropout_prob)(u8)
    c8 = conv_block(u8, n_filters * 2, kernel_size = 3)   #----> 64
    
    u9 = conv_transpose_block(n_filters, kernel_size = 3)(c8)   #----> 32
    u9 = tkl.concatenate([u9, c1])   #----> 64
    u9 = tkl.Dropout(dropout_prob)(u9)
    c9 = conv_block(u9, n_filters , kernel_size = 3)   #----> 32
    
    output = tkl.Conv2D(3, (1,1), activation = 'sigmoid')(c9)
    
    model = tf.keras.Model(inputs = [input_img], outputs = [output])

    return model

In [ ]:
# model = create_model()
model = u_net()
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

In [ ]:
# gaussian_early_stop = EarlyStopping(monitor='loss', patience=3)
# x, y = train_generator
history = model.fit(train_generator,
                              validation_data = val_generator,
                              use_multiprocessing = True,
                              workers = 6, epochs=50)
# , callbacks=[gaussian_early_stop]

# history = model.fit(train_generator, epochs=50)

In [ ]:
model.save('UnetDenoising.h5')

### Evaluate the model: 

In [ ]:
path = '/kaggle/input/stanford-cars-dataset/cars_test/cars_test/'
test_data = list(glob.glob(path + '*.jpg'))
# print(len(test_data))

evaluation_set = test_data[:64]
test_img_path = test_data[104]
test_img = cv2.imread(test_img_path)
test_img = cv2.resize(test_img, (WIDTH, HEIGHT)) / 255
test_img = random_noise(test_img, mode = 'gaussian')
plt.imshow(test_img)
print(test_img.shape)

In [ ]:
model.predict(test_img)

In [ ]:
eval_set = []
for i in range(64):
    img = cv2.imread(evaluation_set[i])
    img = cv2.resize(img, (WIDTH, HEIGHT)) / 255
    img = random_noise(img, mode = 'gaussian')
    eval_set.append(img)
eval_set = np.array(eval_set)

# test_generator = DataGenerator(evaluation_set)
# x, y = test_generator[2]
predicted = model.predict(eval_set)

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
predicted.shape

In [ ]:
# plt.imshow(predicted[10])
predicted_ = cv2.resize(predicted[11], (500, 500))
# fig, ax = plt.subplots(1,3)
prepared_img = cv2.imread(evaluation_set[11]) / 255
prepared_img = cv2.resize(prepared_img, (500, 500))
noisy_orig = random_noise(prepared_img, mode = 'gaussian')
# ax[0].imshow(prepared_img)
# ax[1].imshow(predicted_)
# ax[2].imshow(noisy_orig)


In [ ]:
plt.imshow(cv2.imread(evaluation_set[10]))

In [ ]:
plt.imshow(prepared_img)
plt.title('original image')

In [ ]:
plt.imshow(predicted_)
plt.title('predicted image')

In [ ]:
plt.imshow(noisy_orig)
plt.title('noisy image')